Builds vectors of ***ingroup*** and ***outgroup***.

**Note:** for Sentence-BERT approach, any *A-strategy* other than **really naive** does not make sense. E.g. consider exclusion due to overlap; exclusion of replacements "kriminella invandrare" because "kriminella" is found in replacements of both ingroup and outgroup is ***too strong***, *especially for the out-group in this case* ("kriminella gäng" is the most common ioutgroup replacment of *förortsgäng*). Perhaps, it is still worth testing the different forms of A-strategies as a form of "noise reduction". 

For the SGNS approach, A-strategies arguably are of importance, if we not going to build "sentence vectors" from concatenating word vectors. However, this might point to the limitations of SGNS for thi task, as it is not very good at representing phrases. 

Way forward: build in the possibility to use *A-selection* for SBERT, but focus on results for **really naive**. 

#### Procedure (psuedo code)
```
DWEs = {förortsgäng, återvandring, globalist, berika}
selection_strategies = {really_naive, naive_no_overlap, top1, top3, ...}
models = {sbert_kb, ...}

for DWE in DWEs:
    for strategy in selection_strategies:
        for model in models:

            IN_vectors, OUT_vectors = select(replacement_vectors_of_dwe, strategy)
            avg_IN_vec = mean(IN_vectors)
            avg_OUT_vec = mean(OUT_vectors)

            for year in years:
                dwe_vect_at_year = get_vec(DWE)

                IN_dimension_mean = cosine_similarity(avg_IN_vec, dwe_vect_at_year)
                IN_dimension_pairwise_mean = mean(cosine_similarity(IN_vectors, dwe_vect_at_year))

                OUT_dimension_mean = cosine_similarity(avg_OUT_vec, dwe_vect_at_year)
                OUT_dimension_pairwise_mean = mean(cosine_similarity(OUT_vectors, dwe_vect_at_year))  

                norm_dimension_mean = normalizer(IN_dimension_mean, OUT_dimension_mean)
                norm_dimension_pariwise_mean = normalizer(IN_dimension_pairwise_mean, OUT_dimension_pairwise_mean)

                # e.g. softmax or simply normalize(x, y) = x / (x+y)
```

#### Will get you something like (for each model)...

|DWE            |Selection strategy|Method Dimension|Year<sub>1</sub>|...|Year<sub>*n*</sub>|
|---------------|------------------|----------------|----------------|---|------------------|
|DWE<sub>1</sub>|Really naive      |Mean            |...             |...|...               |
|DWE<sub>1</sub>|Really naive      |Pairwise mean   |...             |...|...               |
|DWE<sub>1</sub>|Really naive      |Normalized      |...             |...|...               |
|DWE<sub>1</sub>|Naive no overlap  |Mean            |...             |...|...               |
|DWE<sub>1</sub>|Naive no overlap  |Pairwise mean   |...             |...|...               |
|DWE<sub>1</sub>|Naive no overlap  |Normalized      |...             |...|...               |
|DWE<sub>1</sub>|Top1              |...             |...             |...|...               |
|...            |...               |...             |...             |...|...               |
|DWE<sub>2</sub>|...               |...             |...             |...|...               |

#### Aditional considerations
* Finetuning
* Correlation with semantic change rates
    * Spearman, Pearson
    * Naive, Rectified
* Correlation with (normalized) frequency
    * Spearman, Pearson

In [99]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from difflib import SequenceMatcher
from pathlib import Path
from sklearn.utils.extmath import softmax
from collections import Counter

In [77]:
def keyness(trg, ref, min_frq = 3, verbose = True): # Consider metric
    
    d = dict()
    
#     trg_tot = sum(trg.values())
#     ref_tot = sum(ref.values())
    trg_tot = len(trg)
    ref_tot = len(ref)
    
    for w in trg.keys():
        if trg[w] < min_frq:
            continue
        if w in ref:
            d[w] = (trg[w] / trg_tot) / (ref[w] / ref_tot) # Odds Ratio (OR)
        else:
            d[w] = np.inf
    
    if verbose:
        for word, trg_freq, keyness  in sorted([(w, trg[w], k) for w, k in d.items()], key = lambda x: x[1], reverse = True)[:20]:
            if word in ref:
                ref_freq = ref[word]
            else:
                ref_freq = 0
            print(f"{word:<20}{trg_freq:<4}{(trg_freq/trg_tot):<6.3f}{ref_freq:<4}{(ref_freq/ref_tot):<6.3f}{keyness:.4}")        
    
    return d
    

In [97]:
def inspect(
    df,            # Replacement Dataframe
    dwe,           # Dog Whistle Expression
    meaning,       # 1 for ingroup, 2 for outgroup
    phase,         # 1 for first phase of data collection, 2 for second phase
    sw = None,     # stopwords
    punct = None,  # remove punctuations
    verbose = True,
    multi = False, # Keep the multi-word units of the replacements
    rel_freq = False # use relative frequncies freq / no. of documents
):
    
    counter = Counter()
    
    if type(df) == pd.DataFrame:
        column = df.loc[df[f"{dwe}_w{phase}_C"] == meaning, f"{dwe}_text_w{phase}"]
    else:
        column = df
    
    for x in column:
        if punct != None:
            for p in punct:
                x = x.replace(p, "")
        x = x.split()
        if sw != None:
            x = [w for w in x if w not in sw]
        
        if multi:
            x = ["_".join(x)]
        
        counter.update(set(x)) # Obs. terms are only counted once per "document"
    
    if rel_freq:
        counter = Counter({w: c/len(column) for w,c in counter.items()})
        
    if verbose:
        for w, f in sorted(counter.items(), key = lambda x: x[1], reverse = True)[:15]:
            print(f"{w:<30}{f}")
        print("-----------------------")
        print("Total no. of types:", len(counter))
    
    

    return counter

In [78]:
def select_A(
    df,             # Replacement Dataframe
    dwe,            # Dog Whistle Expression
    phase = "both", # 1 for first phase of data collection, 2 for second phase, "both" for both
    sw = None,      # stopwords
    punct = None,   # remove punctuations
    k = None,
    min_freq = None,
    min_OR = None,
    empty_intersect = False
):
    
    if type(k) == tuple:
        k_in, k_out = k
    else:
        k_in  = k
        k_out = k
    if type(min_freq) == tuple:
        min_freq_in, min_freq_out = min_freq
    else:
        min_freq_in  = min_freq
        min_freq_out = min_freq
    if type(min_OR) == tuple:
        min_OR_in, min_OR_out = min_OR
    else:
        min_OR_in  = min_OR
        min_OR_out = min_OR
    
    if phase == "both":
        x = pd.concat([
            df.loc[df[f"{dwe}_w{1}_C"] == 1, f"{dwe}_text_w{1}"],
            df.loc[df[f"{dwe}_w{2}_C"] == 1, f"{dwe}_text_w{2}"]
        ]).to_list()
                
        y = pd.concat([
            df.loc[df[f"{dwe}_w{1}_C"] == 2, f"{dwe}_text_w{1}"],
            df.loc[df[f"{dwe}_w{2}_C"] == 2, f"{dwe}_text_w{2}"]
        ]).to_list()

        ingroup = inspect(x, dwe, None, None, sw, punct, verbose = False, rel_freq = True)
        outgroup = inspect(y, dwe, None, None, sw, punct, verbose = False, rel_freq = True)
#         _ = inspect(x, dwe, None, None, sw, punct, multi = True)   
#         _ = inspect(y, dwe, None, None, sw, punct, multi = True)   

        keyness_in2out = keyness(ingroup, outgroup, verbose = False, min_frq = -1)
        keyness_out2in = keyness(outgroup, ingroup, verbose = False, min_frq = -1)
        
    else:    
    
        ingroup = inspect(df, dwe, 1, phase, sw, punct, verbose = False, rel_freq = True)
        outgroup = inspect(df, dwe, 2, phase, sw, punct, verbose = False, rel_freq = True)
#         _ = inspect(df, dwe, 1, phase, sw, punct, multi = True)
#         _ = inspect(df, dwe, 2, phase, sw, punct, multi = True)
        keyness_in2out = keyness(ingroup, outgroup, verbose = False, min_frq = -1)
        keyness_out2in = keyness(outgroup, ingroup, verbose = False, min_frq = -1)
    
    A_in  = [w for w in ingroup.keys()]
    A_out = [w for w in outgroup.keys()]
    #print(A_out)
    
    if empty_intersect:
        A_in  = [w for w in A_in if w not in outgroup.keys()]
        A_out = [w for w in A_out if w not in ingroup.keys()]
        
    if min_freq != None:
        A_in  = [w for w in A_in if ingroup[w] >= min_freq_in]
        A_out = [w for w in A_out if outgroup[w] >= min_freq_out]
    
    #print(A_out)
    
    if min_OR != None:
        A_in  = [w for w in A_in if keyness_in2out[w] >= min_OR_in]
        A_out = [w for w in A_out if keyness_out2in[w] >= min_OR_out] # too strict to have the same threshold for both
        
    #print(A_out)    
    
    if k != None:
        A_in  = [w for w,_ in sorted(ingroup.items(), key = lambda x: x[1], reverse = True) if w in A_in][:k_in]
        A_out = [w for w,_ in sorted(outgroup.items(), key = lambda x: x[1], reverse = True) if w in A_out][:k_out]
    
    
    return A_in, A_out

In [4]:
def matcher(string, A_list, punct = [",", "?", ".", "!", ";"]): 
    # add more sophistication?'
    # punct should match those of Select_A()
    
    match = False

    for p in punct:
        string = string.replace(p, "")
    for w in string.split(" "):
        if w in A_list:
            match = True
            
    return match

In [5]:
def load_replacements(dwe, meaning, rnd, model, data_path):
    
    with open(data_path / dwe / meaning / rnd / "replacements.txt") as f:
        idx_t, text = zip(*[tuple(line.strip("\n").split("\t")) for line in f.readlines()[1:]]) # Obs! skip first line
    with open(data_path / dwe / meaning / rnd / "vectors" / model / "vecs.txt") as f:
        lines = [tuple(line.strip("\n").split("\t")) for line in f.readlines()]
        lines = [(idx, [float(v) for v in vec.split()]) for idx, vec in lines]
        #print(lines[:2])
        idx_v, vectors = zip(*lines)
    
    assert idx_t == idx_v, "Vectors and text are not aligned."
    
#     for t, v in zip(idx_t, idx_v):
#         if t != v:
#             print(t,v)

    #print("MUUUU", vectors[:2])
    
    return [(text, vec) for text, vec in zip(text, vectors)]

In [6]:
def collect_vec(data_path, dwe, Aigt, Aogt, model, rounds = ["first_round", "second_round"]):
    """
    Based on A for the ingroup and the outgroup, collects vectors of the replacements that map to A. 
    Mapping between A and vectors of replacements uses `matcher()`.
    """
    
    igt_vectors = []
    ogt_vectors = []
    
    for meaning in ["ingroup", "outgroup"]:
        for rnd in rounds: # ["first_round", "second_round"] or just one of them
            for replacement, vector in load_replacements(dwe, meaning, rnd, model, data_path):
                if meaning == "ingroup":
                    if matcher(replacement, Aigt): # punctuation
                        igt_vectors.append(vector)
                else:
                    if matcher(replacement, Aogt): # punctuation
                        ogt_vectors.append(vector)
    
    return np.array(igt_vectors), np.array(ogt_vectors)

In [124]:
#     df,            # Replacement Dataframe
#     dwe,           # Dog Whistle Expression
#     phase,         # 1 for first phase of data collection, 2 for second phase, "both" for both
#     sw = None,     # stopwords
#     punct = None,  # remove punctuations
#     k = None,
#     min_freq = None,
#     min_OR = None,
#     empty_intersect = False

def select2vec(mode, dwe, wh_rnds, model, path_dfA, stopwords, punct, data_path, verbose = True):
    """
    Based on a strategy, i.e. `mode`, Select A and returns vectors of replacments that map to A. 
    Uses `select_A()` and `collect_vec()`.
    """

    if mode == "rn":    # Really naive; probably the most sensible for SBERT
        
        igt_vectors = []
        for rnd in wh_rnds:
            _, vecs = zip(*load_replacements(dwe, "ingroup", rnd, model, data_path))
            
            #print(vecs[1]. len(vecs[1]))
            
            igt_vectors.extend(vecs)
        
        ogt_vectors = []
        for rnd in wh_rnds:
            _, vecs = zip(*load_replacements(dwe, "outgroup", rnd, model, data_path))
            ogt_vectors.extend(vecs)  
            
#         print(ogt_vectors[:2])

        return np.array(igt_vectors), np.array(ogt_vectors)
    
    else:
        
        dfA = pd.read_csv(path_dfA, sep="\t") # check parameters
        dfA = dfA.applymap(lambda s: s.lower() if type(s) == str else s)
        
        if wh_rnds == ["first_round"]:
            PHASE = 1
        if wh_rnds == ["second_round"]:
            PHASE = 2
        else:
            PHASE = "both"
        
        if mode == "nno":   # Naive No Overlap
            Aigt, Aogt = select_A(
                df = dfA, 
                dwe = dwe, 
                phase = PHASE, 
                sw = stopwords, 
                punct = punct, 
                empty_intersect = True)

        if mode == "top1":  # Top 1 (no overlap)
            Aigt, Aogt = select_A(
                df = dfA, 
                dwe = dwe,
                phase = PHASE,
                sw = stopwords,
                punct = punct,
                k = 1,
                empty_intersect = True
            )

        if mode == "top3":  # Top 3 (no overlap)
            Aigt, Aogt = select_A(
                df = dfA, 
                dwe = dwe,
                phase = PHASE,
                sw = stopwords,
                punct = punct,
                k = 3,
                empty_intersect = True
            )

        if mode == "ms1":    # Multiple Selection; threshold ... 
            Aigt, Aogt = select_A(
                df = dfA, 
                dwe = dwe,
                phase = PHASE,
                sw = stopwords,
                punct = punct,
                k = 3,
                min_OR = 2.0,
                empty_intersect = False
            )
        
        if verbose:
            if len(Aigt) < 4:
                print("\t", mode)
                print("\tAigt:", ", ".join(Aigt))
                print("\tAogt:", ", ".join(Aogt))
        
        
        if wh_rnds == 1:
            rounds = ["first_round"]
        elif wh_rnds == 2:
            rounds = ["second_round"]
        else: # i.e. wh_rnds == "both"
            rounds = ["first_round", "second_round"]

        return collect_vec(data_path, dwe, Aigt, Aogt, model, rounds)
        #dwe, Aigt, Aogt, model, rounds = ["first_round", "second_round"], data_path



In [8]:
def PairwiseMeanSimilarity(v, v_list):
    
    pairwise = cosine_similarity(v, v_list)
    pairwise_mean = pairwise.mean()
    
    return pairwise_mean
    

In [20]:
def find_corpus_vector(dwe, year, model, path):
    
    with open(path / model / "centroid" / f"{year}.txt") as f:
        lines = [line.strip("\n") for line in f.readlines()]
    
    for line in lines:
        term, vector = tuple(line.split("\t"))
        if term == dwe:
            vector = [float(v) for v in vector.split()]
            return np.array(vector)
    
    return None

In [10]:
def similar_string(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [123]:
def repl_dwe(dwe, rule = None, verbose = True):
    
    if rule != None:
        return rule[dwe]
    else: # infer!
        potential_dwes = ["forortsgang", "aterinvandring", "berikar", "globalister"]
        
        dwe = dwe.split("_")[-1]
        
        best_score = 0
        best_guess = None
        
        for candidate in potential_dwes:
            score = similar_string(dwe, candidate)
            if score > best_score:
                best_score = score
                best_guess = candidate
        
        if verbose:
            print(f"\tInference for {dwe}: {best_guess} (score = {best_score:.2f}).")
        
        return best_guess


In [12]:
class Config:
    def __init__(self):
        self
        
#         self.dwes = dwes
#         self.wh_rounds = rounds
#         self.dfA_path = dfA_path
#         self.strategies = strategies
#         self.years = years
#         self_measures = 
#         self.add_correlations
#         self.model
        

In [ ]:
    
    """
    ...
    
    long = len(dwes) * len(strategies) * len(measures) X 
           |{dwe, strategy, measure, year1, ... year2}|
           # if add_correlations:
           #    |{dwe, strategy, measure, year1, ... year2, r_naive, r_rect, rho_naive, rho_rect, r_fpm, rho_fpm}|
    
    wide = len(dwes) * len(strategies) X 
           |{dwe, strategy}| + len(measure) * len(years)
           # no correlations
           
    
    """

In [111]:
def sbert_builder(config):
    
    results = []
    
    methods = ["I-cnt", "O-cnt", "cnt-ssc", "cnt-smx", "I-pwn", "O-pwn", "pwn-ssc", "pwn-smx"]
    # alternative: make this a config attribute
    
    years = [str(year) for year in range(config.first_year, config.last_year)]
    years.sort()
    
    with open(config.stopwords) as f:
        stopwords = [w.strip("\n") for w in f.readlines()]
    
    for progress, dwe in enumerate(config.dwes, start = 1):
        print(f"Processing {progress} of {len(config.dwes)}: {dwe}.")
        dwe_in_replacement_test = repl_dwe(dwe)
        for strategy in config.strategies:
            
            INGROUPvec, OUTGROUPvec = select2vec(
                mode      = strategy, 
                dwe       = dwe_in_replacement_test, 
                wh_rnds   = config.wh_rounds, 
                model     = config.model, 
                path_dfA  = config.dfA_path, 
                stopwords = stopwords, 
                punct     = config.punct, 
                data_path = config.data_path
            )
            
            #return INGROUPvec
            
            ING_centroid  = INGROUPvec.mean(axis=0)
            OUTG_centroid = OUTGROUPvec.mean(axis=0)

            d = {method: [] for method in methods}

            for year in years:

                vector = find_corpus_vector(dwe, year, config.model, config.path_corpus_vectors)
                
                #print(type(vector))

                if type(vector) != np.ndarray:

                    d["I-cnt"].append(None)
                    d["O-cnt"].append(None)
                    d["cnt-ssc"].append(None)
                    d["cnt-smx"].append(None)
                    d["I-pwn"].append(None)
                    d["O-pwn"].append(None)
                    d["pwn-ssc"].append(None)
                    d["pwn-smx"].append(None)                    

                else:
                    
                    #print(vector.shape, ING_centroid.shape, OUTG_centroid.shape)
                    
#                     i_cnt = cosine_similarity(vector, ING_centroid)[0] 
#                     o_cnt = cosine_similarity(vector, OUTG_centroid)[0]
                    
                    i_cnt = cosine_similarity(vector.reshape(1,-1), ING_centroid.reshape(1,-1))[0][0] 
                    o_cnt = cosine_similarity(vector.reshape(1,-1), OUTG_centroid.reshape(1,-1))[0][0]

                    i_pwn = PairwiseMeanSimilarity(vector.reshape(1, -1), INGROUPvec)
                    o_pwn = PairwiseMeanSimilarity(vector.reshape(1, -1), OUTGROUPvec)
                    
                    #print(i_cnt, o_cnt, i_pwn, o_pwn)

                    d["I-cnt"].append(i_cnt)
                    d["O-cnt"].append(o_cnt) 
                    d["cnt-ssc"].append(i_cnt / (i_cnt + o_cnt))
                    d["cnt-smx"].append(softmax([[i_cnt, o_cnt]])[0][0])

                    d["I-pwn"].append(i_pwn)
                    d["O-pwn"].append(o_pwn) 
                    d["pwn-ssc"].append(i_pwn / (i_pwn + o_pwn))
                    d["pwn-smx"].append(softmax([[i_pwn, o_pwn]])[0][0])

            if config.results_format == "long":
                
                for method in d.keys():
                    line = [dwe, strategy, method]
                    line.extend(d[method])
#                     if config.add_correlations:
#                         r_naive
#                         r_rect
#                         rho_naive
#                         rho_rect
#                         r_fpm
#                         rho_fpm
#                         N                    
                    results.append(line)

            else: # if results_format == "wide"
                line = [dwe, strategy]
                for method in d.keys():
                    line.extend(d[method])
                results.append(line)
    
    if config.results_format == "long":
        features = ["DWE", "A-Strategy", "Method"] + years
        if config.add_correlations:
            additional_headings = ["r_naive", "r_rect", ...]
            features.extend(additional_headings)
        
    
    else: # if wide
        features = ["DWE", "A-Strategy"]
        for method in methods:
            m = [f"{method}_{year}" for year in years]
            features.extend(m)
    
    df = pd.DataFrame(results, columns = features)
    
    df.to_csv(config.results_path)
    
    print("Done!")
    
                    
                
####################################################################################                
                
#                 df.at[i, f"I-cnt_{year}"] = i_cnt
#                 df.at[i, f"O-cnt_{year}"] = o_cnt 
#                 df.at[i, f"cnt-ssc_{year}"] = i_cnt / (i_cnt + o_cnt)
#                 df.at[i, f"cnt-smx_{year}"] = softmax([i_cnt, o_cnt])[0]

#                 df.at[i, f"I-pwn_{year}"] = i_pwn
#                 df.at[i, f"O-pwn_{year}"] = o_pwn 
#                 df.at[i, f"pwn-ssc_{year}"] = i_pwn / (i_pwn + o_pwn)
#                 df.at[i, f"pwn-smx_{year}"] = softmax([i_pwn, o_pwn])[0]
            
#         i += 1 # updates with every iteration of strategy
    

In [64]:
config = Config()

In [125]:
config.first_year = 2000
config.last_year  = 2022
config.dwes       = [
                    "V1_berika",
                    "N1_berikare",
                    "V1_kulturberika",
                    "N1_kulturberikare",
                    "N1_globalist",
                    "A1_globalistisk",
                    "N1_återvandring",
                    "V1_återvandra",
                    #"V1_hjälpa_på_plats",
                    "N1_förortsgäng"
                    ]
config.strategies = ["rn", "nno", "top1", "top3", "ms1"]
config.wh_rounds  = ["first_round", "second_round"]
config.model      = "sentence-bert-swedish-cased"
config.dfA_path   = Path("/home/max/Documents/research/replacement_data/panel_wide_onlyreplace.csv")
config.stopwords  = Path("../../data/utils/stopwords-sv.txt")
config.punct      = [",", "?", ".", "!", ";"]
config.data_path   = Path("/home/max/Results/replacements/data")
config.path_corpus_vectors = Path("/home/max/Results/fb_pol-yearly-bert/")
config.results_format = "long"
config.add_correlations = False
config.results_path = Path("/home/max/Desktop/test.csv")

In [126]:
sbert_builder(config)

Processing 1 of 9: V1_berika.
	Inference for berika: berikar (score = 0.92).
	 top1
	Aigt: förstör
	Aogt: positivt
	 top3
	Aigt: förstör, utnyttjar, försämrar
	Aogt: positivt, ger, gynnar
	 ms1
	Aigt: förstör, utnyttjar, försämrar
	Aogt: tillför, bidrar, förbättrar
Processing 2 of 9: N1_berikare.
	Inference for berikare: berikar (score = 0.93).
	 top1
	Aigt: förstör
	Aogt: positivt
	 top3
	Aigt: förstör, utnyttjar, försämrar
	Aogt: positivt, ger, gynnar
	 ms1
	Aigt: förstör, utnyttjar, försämrar
	Aogt: tillför, bidrar, förbättrar
Processing 3 of 9: V1_kulturberika.
	Inference for kulturberika: berikar (score = 0.63).
	 top1
	Aigt: förstör
	Aogt: positivt
	 top3
	Aigt: förstör, utnyttjar, försämrar
	Aogt: positivt, ger, gynnar
	 ms1
	Aigt: förstör, utnyttjar, försämrar
	Aogt: tillför, bidrar, förbättrar
Processing 4 of 9: N1_kulturberikare.
	Inference for kulturberikare: berikar (score = 0.67).
	 top1
	Aigt: förstör
	Aogt: positivt
	 top3
	Aigt: förstör, utnyttjar, försämrar
	Aogt: posi

In [ ]:
#X = sbert_builder(config)

In [ ]:
X.shape

In [119]:
softmax([[0.13,0.27]])[0][0]

0.4650570548417855

In [ ]:
x = list("ABC")
x.extend(list("DEF"))
x

In [ ]:
np.array([[1,2,3], [3,4,5]]).mean(axis=0)

In [ ]:
df = pd.DataFrame()

for t in ["mamma", "mu"]:
    df["name"] = t
    for m in ["A", "B", "C"]:
        
        for i in [1,2,3]:
            for j in [10,20,30]:
                df.


In [ ]:
df.at[0, "B"] = 1

In [ ]:
df

In [ ]:
pd.DataFrame(None, index=np.arange(5), columns=["name", "A", "B", "C"])

In [ ]:
def foo(mode, config):
    
    rpl_data = pd.read_csv()
    
    dwes = ...
    
    for strategy in A_list:
        
        A_ingroup, A_outgroup = select_A(
            df,            # Replacement Dataframe
            dwe,           # Dog Whistle Expression
            phase,         # 1 for first phase of data collection, 2 for second phase, "both" for both
            sw = None,     # stopwords
            punct = None,  # remove punctuations
            k = None,
            min_freq = None,
            min_OR = None,
            empty_intersect = False
        )
    
    
    if mode == "sgns":
        
        
    if mode == "sbert":
        
        sbert = config
        
        
        
        
    